## Define virality (label)
Meme is labeled 1, viral, if it's score is in the top 5% of memes posted with in the +/- 7 day period around it

In [5]:
import pandas as pd

# data
memes_data = pd.read_json('./Data/Memes/memesfinal.json')
memes_data = memes_data.loc[:, ~memes_data.columns.str.contains('^Unnamed')]
memes_data['date'] = memes_data.id.str[:10]
memes_data['date'] = pd.to_datetime(memes_data['date'])

print(len(memes_data))
memes_data.head()

899525


,id,score,num_comments,over18,url,date,title_raw,ocr_raw,caption_raw,all_text
0,2021.02.15_288,211152,1642,False,https://i.redd.it/n4ildkpurph61.png,2021-02-15,Wait I didn't mean it like that,High schooler: I'd kill for\ncollege tuition\n...,a man with a hat and a beard standing in front...,Wait I didn't mean it like that - High schoole...
1,2020.10.20_959,207206,811,False,https://i.redd.it/gsqgq6uwuau51.jpg,2020-10-20,Don't be mad,"@ renk. 5\n@ 1 Award\n\nI'm a dude, and | fuck...",a couple of white dogs standing next to each o...,Don't be mad - @ renk. 5\n@ 1 Award\n\nI'm a d...
2,2020.01.18_675,207066,1734,False,https://i.redd.it/eg4t9kvlplb41.jpg,2020-01-18,For real tho,...,a cartoon of a cat with a box on its back,For real tho - ...
3,2020.06.09_755,187936,743,False,https://i.redd.it/1wniz8ionv351.jpg,2020-06-09,Right as rain after that,— :\nMy stomach hurts Bo i\n\n/@\nIt’s probabl...,a bunch of legos that are sitting on top of ea...,Right as rain after that - — :\nMy stomach hur...
4,2020.07.08_336,182056,616,False,https://i.redd.it/nrj9smsfek951.jpg,2020-07-08,Licensed Dad Joke,Dads telling jokes at home\n\nDads cling es a ...,a collage of pictures of a dog wearing a birth...,Licensed Dad Joke - Dads telling jokes at home...


In [6]:
# add viral column 
# meme is considered viral if its score in the top 5% of memes posted that week
import numpy as np
from tqdm import tqdm

df_score = memes_data[['id','score','date']]
df_score = df_score.sort_values(by='date', ascending=True)
df_score.reset_index(inplace=True)
df_score['viral'] = np.nan

# save one of each date that occurs in df in order to find date range +/-7 around each date ignoring missing values
meme_dates = []
start = True
for date in df_score.date:
    current = date
    if start == True:
        meme_dates.append(date)
        previous = date
        start = False
    elif current != previous:
        meme_dates.append(date)
        previous = date
        
# get viral feature
for i in tqdm(range(0,len(df_score))):
    # get date range +/- 7 days around date of current record
    cur_date_idx = meme_dates.index(df_score.at[i,'date'])
    if cur_date_idx-7 < 0: # edge case
        begin_range = meme_dates[0]
        end_range = meme_dates[14]
    elif cur_date_idx+7 >= len(meme_dates): # edge case
        begin_range = meme_dates[len(meme_dates)-15]
        end_range = meme_dates[len(meme_dates)-1]
    else:
        begin_range = meme_dates[cur_date_idx-7]
        end_range = meme_dates[cur_date_idx+7]
    # crop df based on this date range and get 95th quantile
    quantile = df_score[df_score.date.between(begin_range, end_range)].score.quantile(0.95)
    # if the score of current record is greater than this append 1 else append 0
    if df_score.at[i,'score'] >= quantile:
        df_score.at[i,'viral'] = 1
    else:
        df_score.at[i,'viral'] = 0

df_score.head()

100%|█████████████████████████████████| 899525/899525 [2:42:21<00:00, 92.34it/s]


,index,id,score,date,viral
0,125964,2018.01.01_71,66,2018-01-01,0.0
1,438307,2018.01.01_142,1,2018-01-01,0.0
2,438305,2018.01.01_53,1,2018-01-01,0.0
3,438304,2018.01.01_129,1,2018-01-01,0.0
4,438303,2018.01.01_122,1,2018-01-01,0.0


In [7]:
# save data
save = df_score[['id','viral']]
save.to_json('./results/processed_features/viral_05.json')
len(save)

899525

Instead of using log upvotes for analysis, we could use a normalized upvotes feature: upvote / mean_upvote_that_week. NOT USED

In [ ]:
# add a normalized upvotes column: upvote/avg_upvote_in_week_around_meme
import numpy as np

# here's a score df to work with (summed score for each date in our time period)
df_score = memes_data[['id','score','date']]
df_score = pd.DataFrame(df_score.groupby(df_score.date).score.sum()); df_score.reset_index(inplace=True)
df_count = memes_data[['id','date']]
df_count = pd.DataFrame(df_count.groupby(df_count.date).id.count()); df_count.reset_index(inplace=True)
df = df_score.merge(df_count,on='date'); df = df.rename({'id':'count_memes','score':'sum_score'}, axis=1)
df.index = pd.DatetimeIndex(df.date)
df = df.drop(['date'],axis=1)

# calculate rolling sum score over week
rolling_score = df.rolling('7D').sum_score.sum() #.resample('W').mean()
rolling_count = df.rolling('7D').count_memes.sum()
df = df.merge(rolling_score, on='date'); df = df.merge(rolling_count, on='date')
df = df.rename({'sum_score_x':'sum_score','count_memes_x':'count_memes','sum_score_y':'rolling_week_score','count_memes_y':'rolling_week_count'}, axis=1)

# function sums the 7 days prior to the date, manually replace score for the first 7 days with that of the 7th day
df.at['2014-01-01', 'rolling_week_count'] = 102; df.at['2014-01-02', 'rolling_week_count'] = 102; df.at['2014-01-03', 'rolling_week_count'] = 102; df.at['2014-01-04', 'rolling_week_count'] = 102; df.at['2014-01-05', 'rolling_week_count'] = 102; df.at['2014-01-06', 'rolling_week_count'] = 102;
df.at['2014-01-01', 'rolling_week_score'] = 3782; df.at['2014-01-02', 'rolling_week_score'] = 3782; df.at['2014-01-03', 'rolling_week_score'] = 3782; df.at['2014-01-04', 'rolling_week_score'] = 3782; df.at['2014-01-05', 'rolling_week_score'] = 3782; df.at['2014-01-06', 'rolling_week_score'] = 3782;
df['rolling_week_avg'] = df['rolling_week_score']/df['rolling_week_count']
df[0:10]